In [ ]:
import torch
import copy
from transformers import CsmForConditionalGeneration, AutoProcessor
from datasets import load_dataset
import tqdm.notebook as tqdm

model_id = "jq/csm-1b-salt"
processor = AutoProcessor.from_pretrained("sesame/csm-1b")
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map="auto")

In [ ]:
speaker_id = "0"
sentences = [
    "I am a language assistant made by Sunbird AI in Uganda.",
    "Nsobola okwogera Oluganda n'ennimi endala.",
    "Atwero loko leb Acholi.",
    "Nimbaasa kugamba Orunyankole.",
    "Apedori eong einer Ateso.",
    "Naweza kuongea Kiswahili.",
]

all_audio = []
for text in tqdm.tqdm(sentences):
    inputs = processor(
        f"[{speaker_id}]{text}", add_special_tokens=True).to("cuda")
    
    audio = model.generate(
        **inputs,
        max_new_tokens=125,
        depth_decoder_temperature=0.6,
        depth_decoder_top_k=0,
        depth_decoder_top_p=0.9,
        temperature=0.8,
        top_k=50,
        top_p=1.0,
        output_audio=True             
    )
    all_audio.append(audio[0].to(torch.float32).cpu().numpy())

In [ ]:
from IPython.display import Audio, display
for i in range(len(all_audio)):
    display(Audio(all_audio[i][:-2000], rate=24000))

In [ ]:
import numpy as np
combined = np.concatenate([audio[:-2000] for audio in all_audio])
display(Audio(combined, rate=24000))